In [5]:
import pandas as pd
import ast
import re

df = pd.read_csv("../data/raw/tmdb_6000/tmdb_6000_movie_dataset.csv")

# Drop index column
df = df.drop(columns=["Unnamed: 0"])
df


,budget,genres,homepage,tmdbId,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.200,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.900,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.300,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.600,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{'id': 818, 'name': 'based on novel'}, {'id':...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.100,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5793,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",https://www.searchlightpictures.com/flamin-hot,626332,"[{'id': 5565, 'name': 'biography'}, {'id': 967...",en,Flamin' Hot,"The inspiring true story of Richard Montañez, ...",53.988000,"[{'id': 61377, 'logo_path': '/6IQmmSszsXcxbew2...","[{'iso_3166_1': 'US', 'name': 'United States o...",2023-03-11,0,99.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The flavor you know. The story you don't.,Flamin' Hot,8.190,410
5794,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,1094319,[],en,The Best Man,Mercenaries seize control of a remote resort h...,47.740000,"[{'id': 76992, 'logo_path': '/mw3j3P8SUQJKLSJQ...",[],2023-04-21,0,93.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,The Best Man,6.167,93
5795,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,998623,[],fr,Drone Games,NaN,27.587000,"[{'id': 178619, 'logo_path': '/wlyh239wcxFSE7K...","[{'iso_3166_1': 'FR', 'name': 'France'}]",2023-10-19,0,0.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,NaN,Drone Games,2.0

In [6]:
def parse_json_list(x):
    """
    Function to convert the json string into python lists
    """
    if pd.isna(x):
        return []
    try:
        data = ast.literal_eval(x)
        if isinstance(data, list):
            names = []
            for item in data:
                if isinstance(item, dict) and "name" in item:
                    names.append(item["name"])
            return names
    except:
        pass
    return []
    

json_cols = ["genres", "keywords", "production_companies", "production_countries", "spoken_languages"]

for col in json_cols:
    df[col + "_list"] = df[col].apply(parse_json_list)


In [13]:
df[['keywords','keywords_list']].head(3)

,keywords,keywords_list
0,"[{'id': 1463, 'name': 'culture clash'}, {'id':...","[culture clash, future, space war, space colon..."
1,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","[ocean, drug abuse, exotic island, east india ..."
2,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...","[spy, based on novel, secret agent, sequel, mi..."


In [15]:
#Extracting primary values in the list

df["primary_genre"] = df["genres_list"].apply(lambda x: x[0] if len(x) > 0 else "Unknown")
df["primary_country"] = df["production_countries_list"].apply(lambda x: x[0] if len(x) > 0 else "Unknown")
df["primary_keyword"] = df["keywords_list"].apply(lambda x: x[0] if len(x) > 0 else "")
df[["primary_genre", "primary_country", "primary_keyword"]].head(3)

,primary_genre,primary_country,primary_keyword
0,Action,United States of America,culture clash
1,Adventure,United States of America,ocean
2,Action,United Kingdom,spy


In [17]:
def clean_text(s):
    if pd.isna(s): 
        return ""
    s = str(s).lower().strip()
    s = re.sub(r"\s+", " ", s)
    return s

text_cols = ["original_title", "overview", "tagline", "title"]

for col in text_cols:
    df[col] = df[col].apply(clean_text)
df[["original_title", "overview", "tagline", "title"]].head(3)

,original_title,overview,tagline,title
0,avatar,"in the 22nd century, a paraplegic marine is di...",enter the world of pandora.,avatar
1,pirates of the caribbean: at world's end,"captain barbossa, long believed to be dead, ha...","at the end of the world, the adventure begins.",pirates of the caribbean: at world's end
2,spectre,a cryptic message from bond’s past sends him o...,a plan no one escapes,spectre


In [21]:
# Convert release_date and create release_year

df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["release_year"] = df["release_date"].dt.year
df["release_year"].head(3)

0    2009.0
1    2007.0
2    2015.0
Name: release_year, dtype: float64

In [24]:
# Creating combined_features = original_title + overview + primary_genre + primary_country + primary_keyword

df["combined_features"] = (
      df["original_title"] + " "
    + df["overview"] + " "
    + df["primary_genre"] + " "
    + df["primary_country"] + " "
    + df["primary_keyword"]
).str.strip()
df["combined_features"].head()

0    avatar in the 22nd century, a paraplegic marin...
1    pirates of the caribbean: at world's end capta...
2    spectre a cryptic message from bond’s past sen...
3    the dark knight rises following the death of d...
4    john carter john carter is a war-weary, former...
Name: combined_features, dtype: object

In [26]:
# Saving cleaned movie file

df.to_csv("../data/processed/movies_clean.csv", index=False)
df.head()

,budget,genres,homepage,tmdbId,keywords,original_language,original_title,overview,popularity,production_companies,...,genres_list,keywords_list,production_companies_list,production_countries_list,spoken_languages_list,primary_genre,primary_country,primary_keyword,release_year,combined_features
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,avatar,"in the 22nd century, a paraplegic marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]","[English, Español]",Action,United States of America,culture clash,2009.0,"avatar in the 22nd century, a paraplegic marin..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,pirates of the caribbean: at world's end,"captain barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],[English],Adventure,United States of America,ocean,2007.0,pirates of the caribbean: at world's end capta...
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,spectre,a cryptic message from bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]","[Français, English, Español, Italiano, Deutsch]",Action,United Kingdom,spy,2015.0,spectre a cryptic message from bond’s past sen...
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",en,the dark knight rises,following the death of district attorney harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],[English],Action,United States of America,dc comics,2012.0,the dark knight rises following the death of d...
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{'id': 818, 'name': 'based on novel'}, {'id':...",en,john carter,"john carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",[Walt Disney Pictures],[United States of America],[English],Action,United States of America,based on novel,2012.0,"john carter john carter is a war-weary, former..."


In [28]:
# quick checks 

from pathlib import Path

p = Path("../data/processed/movies_clean.csv")
print("Exists:", p.exists(), " Size (bytes):", p.stat().st_size if p.exists() else "N/A")

df = pd.read_csv(p)
print("\nShape:", df.shape)
print("\nColumns:", df.columns.tolist())

# show first 3 rows
print("\nFirst 3 rows:")
display(df.head(3))

# key assertions
print("\nNull counts (key cols):")
print(df[['tmdbId','original_title','combined_features','release_date','release_year','runtime']].isna().sum())

# simple distributions
print("\nPrimary genre value counts (top 10):")
print(df['primary_genre'].value_counts().head(10))

print("\nRuntime: min, median, max")
print(df['runtime'].min(), df['runtime'].median(), df['runtime'].max())

print("\nRelease years: min, max, unique count")
print(df['release_year'].min(), df['release_year'].max(), df['release_year'].nunique())

Exists: True  Size (bytes): 10674239

Shape: (5798, 30)

Columns: ['budget', 'genres', 'homepage', 'tmdbId', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'genres_list', 'keywords_list', 'production_companies_list', 'production_countries_list', 'spoken_languages_list', 'primary_genre', 'primary_country', 'primary_keyword', 'release_year', 'combined_features']

First 3 rows:


,budget,genres,homepage,tmdbId,keywords,original_language,original_title,overview,popularity,production_companies,...,genres_list,keywords_list,production_companies_list,production_countries_list,spoken_languages_list,primary_genre,primary_country,primary_keyword,release_year,combined_features
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,avatar,"in the 22nd century, a paraplegic marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['culture clash', 'future', 'space war', 'spac...","['Ingenious Film Partners', 'Twentieth Century...","['United States of America', 'United Kingdom']","['English', 'Español']",Action,United States of America,culture clash,2009.0,"avatar in the 22nd century, a paraplegic marin..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,pirates of the caribbean: at world's end,"captain barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"['Adventure', 'Fantasy', 'Action']","['ocean', 'drug abuse', 'exotic island', 'east...","['Walt Disney Pictures', 'Jerry Bruckheimer Fi...",['United States of America'],['English'],Adventure,United States of America,ocean,2007.0,pirates of the caribbean: at world's end capta...
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,spectre,a cryptic message from bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"['Action', 'Adventure', 'Crime']","['spy', 'based on novel', 'secret agent', 'seq...","['Columbia Pictures', 'Danjaq', 'B24']","['United Kingdom', 'United States of America']","['Français', 'English', 'Español', 'Italiano',...",Action,United Kingdom,spy,2015.0,spectre a cryptic message from bond’s past sen...



Null counts (key cols):
tmdbId               0
original_title       0
combined_features    0
release_date         1
release_year         1
runtime              2
dtype: int64

Primary genre value counts (top 10):
primary_genre
Drama              1316
Comedy             1123
Action              998
Horror              428
Adventure           385
Animation           255
Thriller            242
Crime               219
Fantasy             144
Science Fiction     140
Name: count, dtype: int64

Runtime: min, median, max
0.0 104.0 338.0

Release years: min, max, unique count
1916.0 2023.0 96
